# AIMO Prize 3 - OOP Baseline Submission (v4)

This notebook demonstrates how to use the Object-Oriented Baseline with **Few-Shot Prompting**, **Majority Voting**, and **Parquet Submission**.

## Setup Instructions
1. **Add Utility Script**: Upload `src/kaggle_baseline.py` as a Dataset (e.g., named `aimo-pp3-modules`).
2. **Add Model**: Search for and add the model `Qwen/Qwen2.5-Math-7B-Instruct`.
3. **Attach Competition Data**: Ensure the AIMO 3 competition data is attached.
4. **Run**: Execute the cells below.

In [ ]:
import sys
import os
import glob

# 1. Import the OOP Baseline Module
sys.path.append('/kaggle/input/aimo-pp3-modules')
sys.path.append(os.path.join(os.getcwd(), 'src')) 

try:
    from kaggle_baseline import CompetitionConfig, AIMSolver, MockLLM, LocalVLLM
    print("✅ Successfully imported OOP Baseline modules.")
except ImportError as e:
    print(f"❌ Import Failed: {e}. Make sure the 'aimo-pp3-modules' dataset is attached.")

In [ ]:
# 2. Configure Environment & Model
config = CompetitionConfig()
config.n_repetitions = 1 

print(f"Environment: {'Kaggle' if config.is_kaggle else 'Local'}")

if config.is_kaggle and os.path.exists(config.model_path):
    # Uncomment for Real Model (vLLM)
    # llm = LocalVLLM(config.model_path)
    print("⚠️ Using MockLLM. Uncomment LocalVLLM lines to use the real model.")
    llm = MockLLM()
else:
    llm = MockLLM()

solver = AIMSolver(config, llm)

## Submission Loop (Parquet)

In [ ]:
import pandas as pd

# 3. Locate and Import AIMO API
aimo = None

# 1. Try standard import (Competition environment)
try:
    import aimo
    print("✅ Successfully imported 'aimo' module directly.")
except ImportError:
    pass

# 2. If not found, search for files (Custom dataset or Local fallback)
if not aimo:
    # Search for the API files
    api_files = glob.glob("/kaggle/input/**/aimo_3_inference_server.py", recursive=True)
    api_files += glob.glob("/kaggle/input/**/aimo_3_gateway.py", recursive=True)
    api_files += glob.glob("data/**/aimo_3_inference_server.py", recursive=True) # Local fallback

    if api_files:
        api_path = os.path.dirname(api_files[0])
        if api_path not in sys.path:
            sys.path.append(api_path)
            print(f"Added API path: {api_path}")
        
        try:
            import aimo_3_inference_server as aimo
        except ImportError:
            try:
                import aimo_3_gateway as aimo
            except ImportError:
                print("❌ Found API files but failed to import.")

if aimo:
    try:
        # Try standard init
        if hasattr(aimo, 'make_env'):
            env = aimo.make_env()
        elif hasattr(aimo, 'AIMO3InferenceServer'):
            env = aimo.AIMO3InferenceServer()
        elif hasattr(aimo, 'AIMO3Gateway'):
            env = aimo.AIMO3Gateway()
        else:
            raise ImportError("No suitable environment factory found in API.")
            
        iter_test = env.iter_test()
        print("✅ Connected to AIMO Gateway.")
        
        all_preds = []
        
        for test, sample_submission in iter_test:
            for index, row in test.iterrows():
                result = solver.solve(row['problem'])
                sample_submission.loc[index, 'answer'] = result
            
            all_preds.append(sample_submission.copy())
            env.predict(sample_submission)
            print(f"Submitted batch.")
        
        if all_preds:
            final_df = pd.concat(all_preds)
            final_df.to_parquet('submission.parquet', index=False)
            print("✅ submission.parquet created.")
            
    except Exception as e:
        print(f"❌ Submission Error: {e}")
else:
    print("⚠️ AIMO API not found. Skipping submission loop (Local Mode).")
    # Dummy file for local testing
    if not os.path.exists('submission.parquet'):
        pd.DataFrame({'id': ['test'], 'answer': [0]}).to_parquet('submission.parquet')
        print("Created dummy submission.parquet")